# Bump test recent history

In [ ]:
# Times Square parameters
#day_obs = 20240110 # YYYYMMDD

#day_obs = 20250603 # YYYYMMDD
day_obs = 20250604 # YYYYMMDD


In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio 
await asyncio.sleep(0.01)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

from astropy.time import Time, TimeDelta
from datetime import timedelta

from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index

from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData, getDayObsEndTime, getDayObsStartTime
from lsst.ts.xml.enums.MTM1M3 import BumpTest



In [ ]:
def max_error(errors):
    return np.max([np.max(errors), np.max(errors * -1.0)])

def rms_error(times, errors):
    error = 0.0
    num = 0
    for i, t in enumerate(times):
        if (t > 3.0 and t < 4.0) or (t > 10.0 and t < 11.0):
            num += 1
            error += errors[i]**2
    if num == 0:
        return np.nan
    else:
        return np.sqrt(error / num)
        
async def calc_bumps_and_errors(client, bump, bt_result, follow, timestamp):
    BUMP_TEST_DURATION = 1.0 #14.0  # seconds
    
    measured_forces_times = []
    following_error_values = []
    t_starts = []

    results = bt_result[bt_result[bump] == BumpTest.TESTINGPOSITIVE]
    print("dataframe a analizar")
    print(results)
    for bt_index in results.index:
      print("loop of calc bumps and errors")
      print(bt_index)
      try:
            t_start = Time(bt_index)
      except Exception:
            # si bt_index es str u otro formato:
            t_start = Time(str(bt_index))
      t_starts.append(t_start.isot.split('.')[0])
      t_end = Time(bt_index + timedelta(seconds=BUMP_TEST_DURATION))

      print(f"Consultando datos de {t_start} a {t_end} para {follow}…")

      measured_forces = await client.select_time_series(\
                "lsst.sal.MTM1M3.forceActuatorData", \
                ["primaryCylinderFollowingError", "timestamp"],
                t_start, t_end)
        
      #print("measured_forces")
      #print(len(measured_forces)) 
      #print(measured_forces)
      t0 = measured_forces["timestamp"].values[0]
      measured_forces["timestamp"] -= t0
      #print("measured_forces_for timestamp")
      #print(t0)
      #print(measured_forces["timestamp"].values)
        
      # It is easier/faster to work with arrays
      measured_forces_time = measured_forces["timestamp"].values
      measured_forces_times.append(measured_forces_time)
      following_error_value = measured_forces["primaryCylinderFollowingError"].values
      following_error_values.append(following_error_value)  
      print("llegue a guardar todo en following_error_values")
    times = []
    max_errors = []
    rms_errors = []

    for i in range(len(measured_forces_times)):
        times.append(t_starts[i])
        max_errors.append(max_error(following_error_values[i]))
        rms_errors.append(rms_error(measured_forces_times[i], following_error_values[i]))
    print("ahora salgo de la funcion, luego volvere")    
    return [times, rms_errors, max_errors]

In [ ]:
async def actuator_error(client, fa_id, bt_results):  
       
    # Grab the Force Actuator Data from its ID
    fa_data = force_actuator_from_id(fa_id)

    #bt_result = bt_results[[f"primaryTest{fa_data.index}", f"secondaryTest{fa_data.s_index}", 'timestamp']]

    if fa_data.s_index is not None:
        bt_result = bt_results[[f"primaryTest{fa_data.index}", f"secondaryTest{fa_data.s_index}", f"primaryTestTimestamps{fa_data.index}", 'private_efdStamp']]
    else:
        bt_result = bt_results[[f"primaryTest{fa_data.index}", f"primaryTestTimestamps{fa_data.index}", 'private_efdStamp']]   
    #bt_result = bt_results[bt_results["actuatorId"] == fa_id]
        
    # First the primary forces
    bump = f"primaryTest{fa_data.index}"
    follow = f"primaryCylinderFollowingError{fa_data.index}"
    timestamp = f"primaryTestTimestamps{fa_data.index}"
    print("Primary Forces")
    [ptimes, prms_errors, pmax_errors] = \
        await calc_bumps_and_errors(client, bump, bt_result, follow, timestamp)
    
    # Now the secondary  forces  
    if fa_data.actuator_type.name == "DAA":
        print("Secondary Forces")
        bump = f"secondaryTest{fa_data.s_index}"
        follow = f"secondaryCylinderFollowingError{fa_data.s_index}"
        [stimes, srms_errors, smax_errors] = \
            await calc_bumps_and_errors(client, bump, bt_result, follow, timestamp)
    else:
        stimes = []; srms_errors = []; smax_errors = []

    return [ptimes, prms_errors, pmax_errors, stimes, srms_errors, smax_errors]


In [ ]:
# Create an EFD client instance
client = makeEfdClient()

# Define the start and end time
start_day_obs = day_obs
end_day_obs = day_obs

start = getDayObsStartTime(start_day_obs)
end = getDayObsEndTime(end_day_obs)

try:
    bumps = await client.select_time_series(
        "lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus",
        ['*'], start, end
    )
except Exception as e:
    print("Error reading bumps from EFD:", e)
    bumps = pd.DataFrame()  

# Index for Primary and Secondary force actuator
m1m3_actuator_id_index_table: dict[int, int] = dict(
    [(fa.actuator_id, fa.index) for fa in FATable]
)
m1m3_actuator_id_sindex_table: dict[int, int] = dict(
    [(fa.actuator_id, fa.s_index) for fa in FATable if fa.s_index is not None]
)

'''
ids = []
for index in range(len(FATable)):
    id = FATable[index].actuator_id
    ids.append(id)
'''

In [ ]:
bumps[['primaryTest0', 'primaryTestTimestamps0','private_efdStamp', 'private_identity', 'private_kafkaStamp', 'private_origin', 'private_rcvStamp', 'private_revCode', 'private_seqNum', 'private_sndStamp']]

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,2, figsize=(10,25))

plt.suptitle("Actuator Recent History\nNot plotted -> All green", fontsize=18)
plot_list = [[0,0,"Primary RMS Errors", 0, 1, 5, 10],
             [1,0,"Primary Max Errors", 0, 2, 100, 200],
             [0,1,"Secondary RMS Errors", 3, 4, 7.0, 14.0],
             [1,1,"Secondary Max Errors", 3, 5, 140, 280]]

ns = np.zeros(4) # Number of bad ids per plot
bad_ids = [[],[],[],[]] # List of bad_ids per plot
min_times = np.ones(4) * 1.0E12 # Min/Max times in unix_tai
max_times = np.zeros(4) # Min/Max times in unix_tai
min_time_list = [[],[],[],[]] # Min/Max times in isot
max_time_list = [[],[],[],[]] # Min/Max times in isot

#for id in m1m3_actuator_id_index_table:
for actuator_id, index in m1m3_actuator_id_index_table.items():
    print(f"Actuator ID: {actuator_id} → Index: {index}")

    data = await actuator_error(client, actuator_id, bumps)
    
    for m, [xplot, yplot, title, time_index, error_index, yellow_limit, red_limit] in enumerate(plot_list):
        errors = data[error_index]
        if len(errors) > 0:
            curr_max = np.nanmax(errors)
        else:
            curr_max = 0.0
        if curr_max > yellow_limit:
            bad_ids[m].append(actuator_id)
        else:
            continue

        times = data[time_index]
        actual_times_red = []
        actual_times_yellow = []
        actual_times_green = []
        yaxis_red = []
        yaxis_yellow = []
        yaxis_green = []
    
        for i, time in enumerate(times):
            act_time = Time(time, format='isot').unix_tai
            if act_time < min_times[m]:
                min_times[m] = act_time
                min_time_list[m] = time
            if act_time > max_times[m]:
                max_times[m] = act_time
                max_time_list[m] = time
            
            if errors[i] > red_limit:
                actual_times_red.append(act_time)
                yaxis_red.append(ns[m])
            elif errors[i] > yellow_limit:
                actual_times_yellow.append(act_time)
                yaxis_yellow.append(ns[m])
            else:
                actual_times_green.append(act_time)
                yaxis_green.append(ns[m])

        axs[xplot][yplot].set_title(title)
        axs[xplot][yplot].scatter(actual_times_green, yaxis_green, color='green')
        axs[xplot][yplot].scatter(actual_times_yellow, yaxis_yellow, color='gold')
        axs[xplot][yplot].scatter(actual_times_red, yaxis_red, color='red')
        ns[m] += 1
        
for m, [xplot, yplot, title, time_index, error_index, yellow_limit, red_limit] in enumerate(plot_list):
    axs[xplot][yplot].set_yticks(list(range(len(bad_ids[m]))), bad_ids[m])
    axs[xplot][yplot].set_xticks([min_times[m], max_times[m]],[min_time_list[m], max_time_list[m]], rotation=10)

plt.show()